In [ ]:
import os
import numpy as np
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

In [ ]:
# Mounting Google Drive in Colab
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Direktori dataset Seasonal Color
seasonal_color_directory = '/content/drive/MyDrive/smartfit/dataset/seasonal_color'

In [ ]:
# Tampilkan distribusi dataset
for category in os.listdir(seasonal_color_directory):
    category_path = os.path.join(seasonal_color_directory, category)
    if os.path.isdir(category_path):
        print(f"{category}: {len(os.listdir(category_path))} images")

autumn: 179 images
winter: 114 images
spring: 169 images
summer: 205 images


In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

In [ ]:
# Training Data Generator
train_generator = datagen.flow_from_directory(
    seasonal_color_directory,
    target_size=(128, 128),
    batch_size=16,  # Smaller batch size
    class_mode='sparse',
    subset='training'
)

Found 533 images belonging to 4 classes.


In [ ]:
# Validation Data Generator
val_generator = datagen.flow_from_directory(
    seasonal_color_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 130 images belonging to 4 classes.


In [ ]:
# Hitung class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

NameError: name 'compute_class_weight' is not defined

In [ ]:
# Load Pre-trained Model (EfficientNet)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

In [ ]:
# Tambahkan top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile model
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [ ]:
# Training model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
# Unfreeze sebagian layer untuk fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Freeze sebagian besar layer
    layer.trainable = False

In [ ]:
# Compile ulang model
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Fine-tuning
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
# Final evaluation
loss, accuracy = model.evaluate(val_generator)
print(f"Final Validation Loss: {loss}")
print(f"Final Validation Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Simpan model
model.save('/content/drive/MyDrive/smartfit/models/seasonal_color_model.h5')
print("Model saved successfully!")

In [ ]:
import matplotlib.pyplot as plt

# Visualisasi hasil pelatihan
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()